In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12,12]


def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [3]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(trajectories, n, f, x_min):
    fig, ax = plt.subplots()
    colors = ['blue']
    
    def step(t):
        ax.cla()
        ax.plot([x_min[0]], [x_min[1]], 'o', color='green')
        ax.add_patch(plt.Rectangle((0, 0), 1, 1))
        # Level contours
        delta = 0.025
        x = np.arange(-3, 3, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        # print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [0.5, 3, 7.5], colors=['blue', 'purple', 'red'])

        for traj, color in zip(trajectories, colors):
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], color=color)
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], 'o', color=color)

        fix_scaling(ax)
        ax.axis('off')

    plt.close()
    return FuncAnimation(fig, step, frames=range(n), interval=600)

In [4]:
def f(x):
    return 3 * x[0] ** 2 + 22 * (x[0] - x[1]) ** 2 - x[0] - 2 * x[1]


def f_grad(x):
    return np.array([50 * x[0] - 44 * x[1] - 1,
                     - 44 * x[0] + 44 * x[1] - 2], dtype=np.float32)


def f_hessian():
    return np.array([[50, -44],
                     [-44, 44]], dtype=np.float32)


def hessian_eigen_values():
    return list(np.linalg.eig(f_hessian()))[0]

In [5]:
def closest_point(a, b, x):
    if x < a:
        return a
    if a <= x <= b:
        return x
    return b


def projection(x):
    vectorized = np.vectorize(lambda xi: closest_point(0, 1, xi))
    y = vectorized(x)
    return y


def projected_gradient_descent(x_start, gradient):
    alpha = 1e-3
    traj_projected = [x_start.copy()]
    cur_x = x_start.copy()
    for i in range(NUMBER_OF_STEPS):
        cur_x = projection(cur_x - alpha * gradient(cur_x))
        traj_projected.append(cur_x.copy())

    print(traj_projected[-1])
    return [traj_projected, 'Projected gradient descent']

In [6]:
def solve_equation():
    zeros = np.zeros(2)
    x_min = np.linalg.solve(f_hessian(), -f_grad(zeros))
    print(x_min)
    # print(f(x_min))
    return x_min, f(x_min)

In [17]:
NUMBER_OF_STEPS = 30

In [18]:
trajectories = []
x_start = np.array([0.1, 0.7])
trajectories.append(projected_gradient_descent(x_start, f_grad))
x_min, f_min = solve_equation()

[ 0.38325831  0.45366641]
[ 0.5         0.54545456]


In [19]:
base_animation = animate_trajectory(trajectories, NUMBER_OF_STEPS, f, x_min)
HTML(base_animation.to_html5_video())